In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential, clone_model
from keras.layers import Dense, Dropout, Activation, Flatten
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import keras_tuner as kt


In [2]:
um_data = pd.read_csv("../Data/UM_data_top_100.csv")

In [3]:
um_data['log_mph'] = np.log(um_data['mph'])

In [4]:
um_data['Athlete gender'] = um_data['Athlete gender'].map({'F': 0, 'M': 1})

In [5]:
um_data.rename(columns={'Athlete gender': "Gender"}, inplace=True)
um_data.rename(columns={'Average state elevation (feet)': "State_elevation"}, inplace=True)
um_data.rename(columns={'true age': "true_age"}, inplace=True)
um_data.rename(columns={'Distance (miles)': "distance"}, inplace=True)

In [6]:
um_data = um_data[um_data['true_age'] < 63]

In [7]:
um_data = um_data[um_data['true_age'] > 17]

In [8]:
um_data.describe()

,Unnamed: 0,Year of event,Event number of finishers,Athlete year of birth,Gender,Athlete ID,true_age,Highest Elevation (feet),Elevation Gain (feet),State_elevation,distance,Hours Ran,mph,log_mph
count,4.109950e+05,410995.000000,410995.000000,410995.000000,410995.000000,4.109950e+05,410995.000000,410995.000000,410995.000000,410995.000000,410995.000000,410995.000000,410995.000000,410995.000000
mean,7.881171e+05,2009.251558,288.374236,1967.784949,0.724405,5.972267e+05,41.466609,3316.694393,6659.579647,2309.769187,46.973257,10.873245,4.718137,1.526477
std,3.802203e+05,9.060303,215.115558,13.316914,0.446814,4.816509e+05,9.465575,3422.289391,5649.298984,1830.432972,22.825843,7.160621,1.062068,0.223930
min,8.570000e+02,1963.000000,1.000000,1912.000000,0.000000,5.500000e+01,18.000000,200.000000,100.000000,489.000000,26.200000,2.863611,1.567142,0.449254
25%,5.532145e+05,2004.000000,147.000000,1958.000000,0.000000,1.408000e+05,35.000000,1172.000000,2500.000000,814.000000,31.000000,6.391111,3.975208,1.380077
50%,8.078550e+05,2011.000000,223.000000,1968.000000,1.000000,4.500030e+05,41.000000,1619.000000,5456.000000,1700.000000,36.000000,8.386111,4.601682,1.526422
75%,1.076104e+06,2016.000000,355.000000,1978.000000,1.000000,1.071039e+06,48.000000,5000.000000,8000.000000,3048.000000,50.000000,11.673889,5.334756,1.674243
max,1.408415e+06,2022.000000,1329.000000,2004.000000,1.000000,1.641167e+06,62.000000,14048.000000,33197.000000,6800.000000,100.000000,49.148056,12.326656,2.511764


In [ ]:
plt.figure(figsize=(12, 8))
sns.boxenplot(data=um_data[['true_age', 'distance', 'mph']])
plt.show()

In [9]:
X = um_data[['Gender', 'State_elevation', 'true_age', 'distance']].values
y = um_data['mph'].values

In [11]:
y = y.reshape(-1, 1) if len(y.shape) == 1 else y

In [12]:
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_poly = poly.fit_transform(X)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [14]:
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

In [15]:
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [16]:
# Define a more complex model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1, activation='linear')
])

In [17]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mae"])

In [19]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(tf.keras.layers.Dense(units=hp_units, activation="relu"))
    
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(tf.keras.layers.Dropout(rate=hp_dropout))
    
    model.add(tf.keras.layers.Dense(1, activation='linear'))
    
    model.compile(optimizer="adam", loss='mean_squared_error', metrics=['mae'])
    return model
 
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='hyperparamter_tuning'
 )

tuner.search(X_train_scaled, y_train_scaled, epochs=50, validation_split=0.2)
    

Trial 83 Complete [00h 02m 10s]
val_loss: 0.7503249049186707

Best val_loss So Far: 0.6736776232719421
Total elapsed time: 00h 37m 06s


In [21]:
best_model = tuner.get_best_models(num_models=1)[0]

In [24]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_index, val_index in kf.split(X_train_scaled):
    X_train_kfold, X_val_kfold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_kfold, y_val_kfold = y_train_scaled[train_index], y_train_scaled[val_index]
    
    # Reshape y arrays to ensure they are 2D
    if len(y_train_kfold.shape) == 1:
        y_train_kfold = y_train_kfold.reshape(-1, 1)
    if len(y_val_kfold.shape) == 1:
        y_val_kfold = y_val_kfold.reshape(-1, 1)

    # Compile the best model before each fold training
    best_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Train the model on the K-fold training data
    best_model.fit(X_train_kfold, y_train_kfold, epochs=50, batch_size=32, verbose=0, validation_data=(X_val_kfold, y_val_kfold))

    # Evaluate the model on the K-fold validation data
    loss, mae = best_model.evaluate(X_val_kfold, y_val_kfold)
    cv_scores.append(mae)
    print(f'KFold Validation loss: {loss}, MAE: {mae}')

print(f'Mean CV MAE: {np.mean(cv_scores)}')



2055/2055 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - loss: 0.6539 - mae: 0.6241
KFold Validation loss: 0.6551622152328491, MAE: 0.6255162954330444
2055/2055 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step - loss: 0.6293 - mae: 0.6138
KFold Validation loss: 0.6336243748664856, MAE: 0.6159797310829163
2055/2055 ━━━━━━━━━━━━━━━━━━━━ 1s 242us/step - loss: 0.6400 - mae: 0.6203
KFold Validation loss: 0.6423247456550598, MAE: 0.6219559907913208
2055/2055 ━━━━━━━━━━━━━━━━━━━━ 1s 244us/step - loss: 0.6614 - mae: 0.6354
KFold Validation loss: 0.6596304774284363, MAE: 0.6346040368080139
2055/2055 ━━━━━━━━━━━━━━━━━━━━ 1s 245us/step - loss: 0.6356 - mae: 0.6158
KFold Validation loss: 0.639180064201355, MAE: 0.6176964044570923
Mean CV MAE: 0.6231504917144776


In [27]:

n_models = 5  # Number of models in the ensemble
models = []

# Ensure y_train_scaled is reshaped to 2D
if len(y_train_scaled.shape) == 1:
    y_train_scaled = y_train_scaled.reshape(-1, 1)

for i in range(n_models):
    # Clone the best model to ensure all models have the same architecture and parameters
    model = clone_model(best_model)
    model.set_weights(best_model.get_weights())

    # Compile the cloned model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    # Train the cloned model on the entire training data
    model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32, verbose=0)
    
    # Append the trained model to the models list
    models.append(model)
    print(f'Model {i+1} trained.')

print("All models trained.")


Model 1 trained.
Model 2 trained.
Model 3 trained.
Model 4 trained.
Model 5 trained.
All models trained.


In [120]:
# Ensure model summary and configurations are correct
for model in models:
    print(model.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 480)            │         5,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           481 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,725 (73.15 KB)

 Trainable params: 6,241 (24.38 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 18,725 (73.15 KB)

 Trainable params: 6,241 (24.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,484 (48.77 KB)

None


In [122]:
import numpy as np

def manual_feature_importance(models, X_sample, y_sample):
    baseline_predictions = np.mean([model.predict(X_sample) for model in models], axis=0)
    feature_importances = []
    
    for i in range(X_sample.shape[1]):
        X_sample_temp = X_sample.copy()
        np.random.shuffle(X_sample_temp[:, i])
        shuffled_predictions = np.mean([model.predict(X_sample_temp) for model in models], axis=0)
        importance = np.mean(np.abs(baseline_predictions - shuffled_predictions))
        feature_importances.append(importance)
    
    return feature_importances

# Scale the sample
X_sample = X_train[:100]
X_sample_scaled = scaler_X.transform(poly.transform(X_sample[:, :4]))  # Use only the first 4 features and transform
y_sample = y_train[:100]

# Calculate feature importances manually
importances = manual_feature_importance(models, X_sample_scaled, y_sample)
print("Manual Feature Importances: ", importances)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 

In [125]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# Ensure poly and scaler_X are already fitted on the training data

def prepare_data_full(gender, true_age, state_elevation, distance):
    """
    Function to prepare data including polynomial features and interaction terms.
    
    Parameters:
    gender (int): Gender
    true_age (int): True age
    state_elevation (int): State elevation
    distance (int): Distance
    
    Returns:
    np.array: Scaled polynomial transformed data
    """
    new_data = np.array([[gender, true_age, state_elevation, distance]])
    
    # Print original input data
    print(f'Original input data: {new_data}')
    
    # Transform the new data to generate polynomial features
    new_data_poly = poly.transform(new_data)
    print(f'Polynomial Transformed Data: {new_data_poly}')
    
    # Scale the transformed new data
    new_data_scaled = scaler_X.transform(new_data_poly)
    print(f'Scaled Transformed Data: {new_data_scaled}')
    
    return new_data_scaled

# Example new data for prediction
new_data_example = prepare_data_full(1, 27, 5600, 32)

# Ensure new_data_scaled has the correct number of features
print(f'New Data Scaled Shape: {new_data_example.shape}')

# Predict using the ensemble of models
predicted_mph_scaled = ensemble_predict(new_data_example, models)

# Transform the prediction back to the original scale
predicted_mph = scaler_y.inverse_transform(predicted_mph_scaled.reshape(-1, 1))

# Print final predictions
print(f'Predicted race pace (MPH): {predicted_mph[0][0]}')


Original input data: [[   1   27 5600   32]]
Polynomial Transformed Data: [[1.000e+00 1.000e+00 2.700e+01 5.600e+03 3.200e+01 2.700e+01 5.600e+03
  3.200e+01 1.512e+05 8.640e+02 1.792e+05]]
Scaled Transformed Data: [[ 0.00000000e+00  6.16741941e-01 -1.24690073e+00  5.86932323e+02
  -6.55688957e-01 -8.68827540e-01  2.72969561e+02 -1.01100240e-01
   6.95675021e-01 -8.27248981e-01  1.62364788e+02]]
New Data Scaled Shape: (1, 11)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Predicted race pace (MPH): -467.3152160644531


In [126]:
# Compare with known data transformation
known_sample_idx = 0
known_data = X_train[known_sample_idx][:4]  # Ensure only the first four features are used
poly_sample_known = poly.transform([known_data])
scaled_poly_sample_known = scaler_X.transform(poly_sample_known)

# Print comparisons
print(f'Original Known Data: {known_data}')
print(f'Polynomial Transformed Known Data: {poly_sample_known}')
print(f'Scaled Polynomial Features of Known Sample: {scaled_poly_sample_known}')


Original Known Data: [1.00e+00 1.00e+00 1.05e+03 4.80e+01]
Polynomial Transformed Known Data: [[1.00e+00 1.00e+00 1.00e+00 1.05e+03 4.80e+01 1.00e+00 1.05e+03 4.80e+01
  1.05e+03 4.80e+01 5.04e+04]]
Scaled Polynomial Features of Known Sample: [[ 0.00000000e+00  6.16741941e-01 -1.26110428e+00  1.06492501e+02
   4.58592156e-02 -8.82614002e-01  4.99740987e+01  4.42454103e-01
  -1.19210371e+00 -8.32985450e-01  4.43770706e+01]]
